<a href="https://colab.research.google.com/github/SmartGridsML/30daysML/blob/main/bilal_wav2vec2_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!add-apt-repository -y ppa:jonathonf/ffmpeg-4 && apt update && apt install -y ffmpeg
!pip install --quiet datasets transformers evaluate huggingface_hub jiwer

Repository: 'deb https://ppa.launchpadcontent.net/jonathonf/ffmpeg-4/ubuntu/ jammy main'
Description:
Backport of FFmpeg 4 and associated libraries. Now includes AOM/AV1 support!

FDK AAC is not compatible with GPL and FFmpeg can't be redistributed with it included. Please don't ask for it to be added to this public PPA.

---

PPA supporters:

BigBlueButton (https://bigbluebutton.org)

---

Donate to FFMPEG: https://ffmpeg.org/donations.html
Donate to Debian: https://www.debian.org/donations
Donate to this PPA: https://ko-fi.com/jonathonf
More info: https://launchpad.net/~jonathonf/+archive/ubuntu/ffmpeg-4
Adding repository.
Adding deb entry to /etc/apt/sources.list.d/jonathonf-ubuntu-ffmpeg-4-jammy.list
Adding disabled deb-src entry to /etc/apt/sources.list.d/jonathonf-ubuntu-ffmpeg-4-jammy.list
Adding key to /etc/apt/trusted.gpg.d/jonathonf-ubuntu-ffmpeg-4.gpg with fingerprint 4AB0F789CBA31744CC7DA76A8CF63AD3F06FC659
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http:/

In [ ]:
from huggingface_hub import login

login()

#Load & Prepare an Audio Dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset("Shirali/ISSAI_KSC_335RS_v_1_1", split="train",streaming=True, use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Resolving data files:   0%|          | 0/40 [00:00<?, ?it/s]

In [ ]:
print(next(iter(dataset)))

{'uttID': '5ed899a15482d', 'deviceID': 5, 'text': 'ілгеріде көшіп қонып үзілмей тұрғанда бұл даланы бұрынғылар іш жақ деп атаған екен', 'audio': {'path': '5ed899a15482d.flac', 'array': array([ 0.00000000e+00,  0.00000000e+00,  3.05175781e-05, ...,
       -1.58996582e-02,  1.22375488e-02,  2.07824707e-02]), 'sampling_rate': 16000}}


In [ ]:
import IPython.display as ipd

sample = next(iter(dataset))
audio = sample["audio"]

print(sample["text"])
ipd.Audio(data=audio["array"], autoplay=True, rate=audio["sampling_rate"])

ілгеріде көшіп қонып үзілмей тұрғанда бұл даланы бұрынғылар іш жақ деп атаған екен


## Pre-Process the Dataset

In [ ]:
# We can set the sampling rate of our audio dataset through the cast_column method.
# This doesn't change the dataset in-place, but resamples the dataset on
# the fly the first time a sample is loaded.
from datasets import Audio

#dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

##Create Wav2Vec2CTCTokenizer

In [ ]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower()
    return batch

In [ ]:
dataset = dataset.map(remove_special_characters)

In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
print(dataset.column_names)

None


In [ ]:
vocabs = dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dataset.column_names["train"])

TypeError: 'NoneType' object is not subscriptable

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer